In [ ]:
import os
import asyncio
import json
import pandas as pd
from crawl4ai import AsyncWebCrawler, BrowserConfig, CrawlerRunConfig, LLMConfig, CacheMode
from crawl4ai.extraction_strategy import LLMExtractionStrategy
from openai import OpenAI
import numpy as np
from pydantic import BaseModel, Field
import PyPDF2

from secret import openai_key

In [ ]:
id = 'volkswagen'
with open('config.json', 'r') as f:
    config = json.load(f)[id]

In [ ]:
from pathlib import Path
root_path = f"data\\{id}"
Path(root_path).mkdir(parents=True, exist_ok=True)

In [ ]:
client = OpenAI(api_key=openai_key)

# Annual Report DiscoverIdentification

In [ ]:
data = pd.read_csv(f'{root_path}\\links.csv', index_col=0).to_dict('records')
data

In [ ]:
#special_instructions = "This company often names its annual reports like reference document or universal registration document. Prefer these over files that are named like annual report."
#special_instructions = "In case of duplicates, prefer GAAP adjusted."
special_instructions = config['annual_reports']['special_instructions']

In [ ]:
system_prompt = f"""
You are a utility which picks out the relevant annual reports from a list of URLs.
"""

prompt = f"""
Get the annual reports from a list of candidate PDF URLs. There must be only one
annual report per year. Are are looking for the one with financial statements.

Make sure the year is correct. If a date is present in a URL or filename, it
might be an upload date. To resolve this, note that the annual report is always
uploaded after the year its for.

Sort the output by year descending.

If there are missing years between the newest and oldest year, there's a high
likelihood that data isn't missing. Try to ensure those gaps are filled.

Special instructions: {special_instructions}
Dataset: {json.dumps(data)}
"""

schema = {
    "type": "object",
    "properties": {
        "annual_reports": {
            "type": "array",
            "items": {
                "type": "object",
                "properties": {
                    "year": {
                        "type": "integer",
                        "description": "The fiscal year of the report."
                    },
                    "filename": {
                        "type": "string",
                        "description": "The name of the PDF file."
                    },
                    "url": {
                        "type": "string",
                        "description": "The URL linking to the annual report PDF."
                    }
                },
                "required": ["year", "filename", "url"],
                "additionalProperties": False
            }
        }
    },
    "required": ["annual_reports"],
    "additionalProperties": False
}

response = client.responses.create(
    model="o3-mini",
    input=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": prompt}
    ],
    text={
        "format": {
            "type": "json_schema",
            "name": "annual_report",
            "schema": schema,
            "strict": True
        }
    }
)

In [ ]:
annual_reports = pd.DataFrame(json.loads(response.output_text)['annual_reports']).set_index('year')
annual_reports

In [ ]:
suspicious_years = []
for year in range(1980, 2025):
    if (str(year) in json.dumps(data)) and (not year in annual_reports.index.values):
        suspicious_years.append(year)

In [ ]:
if len(suspicious_years) > 0:
    print(f"Warning: years found in URLs without annual reports: {suspicious_years}")

In [ ]:
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

In [ ]:
import urllib
for y, row in annual_reports.iterrows():
    report_path = f"{root_path}\\{y}"
    Path(report_path).mkdir(parents=True, exist_ok=True)
    urllib.request.urlretrieve(row['url'], report_path+f"\\{row['filename']}")

# Data Parsing

In [ ]:
annual_reports = pd.read_csv(f"{root_path}\\pdfs.csv", index_col=0)

In [ ]:
y = 2018
report_path = f"{root_path}\\{y}"
report_name = annual_reports.loc[y]['filename']
pdf_path = f"{report_path}\\{report_name}"

In [ ]:
filesize = os.path.getsize(pdf_path)/1024**2
with open(pdf_path, "rb") as infile:
    reader = PyPDF2.PdfReader(infile)
    num_pages = len(reader.pages)
if filesize > 32 or num_pages > 100:
    print("File too large for a single request. Use split method.")

In [ ]:
file = client.files.create(
    file=open(pdf_path, "rb"),
    purpose="user_data"
)

In [ ]:
special_instructions = config['report_parser']['special_instructions']

instruction = f"""

From the given annual report, extract the balance sheet, income statement, and 
cash flow statement data for the current year. The current year is {y}.

Your response should simply be a JSON dictionary, ready to load into json.loads.
The top level dict keys must be balance_sheet, income_statement, and
cash_flow_statement. The values for these keys is another dict, where the items
are the relevant data you find. Do not nest another dictionary beyond this
level. Make sure there are no duplicate keys. I would like dict keys in
lowercase_with_underscores.

Special instructions: {special_instructions}
"""
response = client.responses.create(
    model="o1",
    input=[
        {
            "role": "user",
            "content": [
                {
                    "type": "input_file",
                    "file_id": file.id,
                },
                {
                    "type": "input_text",
                    "text": instruction,
                },
            ]
        }
    ]
)

In [ ]:
final_dict = json.loads(response.output_text)
final_dict['year'] = y

In [ ]:
final_dict

In [ ]:
final_dict['year'] = y
with open(f"{report_path}\\output.json", 'w') as f:
    json.dump(final_dict, f)

## Splitting Large PDFs

In [ ]:
balance_sheet_pages = []
income_statement_pages = []
cash_flow_statement_pages = []

with open(pdf_path, "rb") as infile:
    reader = PyPDF2.PdfReader(infile)
    num_pages = len(reader.pages)
    
    for i in range(num_pages):
        page = reader.pages[i]
        text = page.extract_text()
        if text and 'balance sheet' in text.lower():
            for j in range(i, min(i + 3, num_pages)):
                if j not in balance_sheet_pages:
                    balance_sheet_pages.append(j)
        if text and 'income statement' in text.lower():
            for j in range(i, min(i + 3, num_pages)):
                if j not in income_statement_pages:
                    income_statement_pages.append(j)
        if text and 'cash flow statement' in text.lower():
            for j in range(i, min(i + 3, num_pages)):
                if j not in income_statement_pages:
                    cash_flow_statement_pages.append(j)

In [ ]:
def save_reduced_pdf(input_pdf_path, output_pdf_path, page_numbers):
    """
    Create a reduced PDF from the input_pdf_path containing only the pages listed in page_numbers.
    
    Parameters:
        input_pdf_path (str): Path to the original PDF.
        output_pdf_path (str): Path where the reduced PDF will be saved.
        page_numbers (list): A list of page numbers (0-indexed) to include in the new PDF.
    """
    with open(input_pdf_path, "rb") as infile:
        reader = PyPDF2.PdfReader(infile)
        writer = PyPDF2.PdfWriter()
        
        # Iterate through the list of page numbers
        for page_num in page_numbers:
            # Check if the page number is within the range of available pages
            if page_num < len(reader.pages):
                writer.add_page(reader.pages[page_num])
            else:
                print(f"Page {page_num + 1} is out of range. Skipping.")
        
        # Save the reduced PDF
        with open(output_pdf_path, "wb") as outfile:
            writer.write(outfile)
        print(f"Reduced PDF saved as: {output_pdf_path}")

In [ ]:
balance_sheet_pdf = f"{report_path}\\_balance_sheet_{report_name}"
income_statement_pdf = f"{report_path}\\_income_statement_{report_name}"
cash_flow_statement_pdf = f"{report_path}\\_cash_flow_statement_{report_name}"

In [ ]:
save_reduced_pdf(pdf_path, balance_sheet_pdf, balance_sheet_pages)
save_reduced_pdf(pdf_path, income_statement_pdf, income_statement_pages)
save_reduced_pdf(pdf_path, cash_flow_statement_pdf, cash_flow_statement_pages)

In [ ]:
balance_sheet_file = client.files.create(
    file=open(balance_sheet_pdf, "rb"),
    purpose="user_data"
)

In [ ]:
instruction = f"""
From the given annual report, extract the balance sheet data for the current
year. The current year is {y}.

Your response should simply be a JSON dictionary, ready to load into json.loads.
Dict items are the relevant data you find. Make sure there are no duplicate
keys. Do not give a nested dictionary. I would like dict keys in
lowercase_with_underscores.

Special instructions: {special_instructions}
"""
response = client.responses.create(
    model="o1",
    input=[
        {
            "role": "user",
            "content": [
                {
                    "type": "input_file",
                    "file_id": balance_sheet_file.id,
                },
                {
                    "type": "input_text",
                    "text": instruction,
                },
            ]
        }
    ]
)
balance_sheet_dict = json.loads(response.output_text)

In [ ]:
balance_sheet_dict

In [ ]:
income_statement_file = client.files.create(
    file=open(income_statement_pdf, "rb"),
    purpose="user_data"
)

In [ ]:
instruction = f"""
From the given annual report, extract the income statement data for the current
year. The current year is {y}.

Your response should simply be a JSON dictionary, ready to load into json.loads.
Dict items are the relevant data you find. Make sure there are no duplicate
keys. Do not give a nested dictionary. I would like dict keys in
lowercase_with_underscores.

Special instructions: {special_instructions}
"""
response = client.responses.create(
    model="o1",
    input=[
        {
            "role": "user",
            "content": [
                {
                    "type": "input_file",
                    "file_id": income_statement_file.id,
                },
                {
                    "type": "input_text",
                    "text": instruction,
                },
            ]
        }
    ]
)
income_statement_dict = json.loads(response.output_text)

In [ ]:
income_statement_dict

In [ ]:
cash_flow_statement_file = client.files.create(
    file=open(cash_flow_statement_pdf, "rb"),
    purpose="user_data"
)

In [ ]:
instruction = f"""
From the given annual report, extract the cash flow statement data for the
current year. The current year is {y}.

Your response should simply be a JSON dictionary, ready to load into json.loads.
Dict items are the relevant data you find. Make sure there are no duplicate
keys. Do not give a nested dictionary. I would like dict keys in
lowercase_with_underscores.

Special instructions: {special_instructions}
"""
response = client.responses.create(
    model="o1",
    input=[
        {
            "role": "user",
            "content": [
                {
                    "type": "input_file",
                    "file_id": cash_flow_statement_file.id,
                },
                {
                    "type": "input_text",
                    "text": instruction,
                },
            ]
        }
    ]
)
cash_flow_statement_dict = json.loads(response.output_text)

In [ ]:
cash_flow_statement_dict

In [ ]:
final_dict = {
    'year': y,
    'balance_sheet': balance_sheet_dict,
    'income_statement': income_statement_dict,
    'cash_flow_statement': cash_flow_statement_dict
}

In [ ]:
with open(f"{report_path}\\output.json", 'w') as f:
    json.dump(final_dict, f)

# Consolidating

In [ ]:
balance_sheet = []
income_statement = []
cash_flow_statement = []
for y in annual_reports.index:
    output_file = f"{root_path}\\{y}\\output.json"
    if os.path.exists(output_file):
        with open(output_file) as f:
            statement = json.load(f)

            statement['balance_sheet'].update({'year':y})
            balance_sheet.append(statement['balance_sheet'])

            statement['income_statement'].update({'year':y})
            income_statement.append(statement['income_statement'])

            statement['cash_flow_statement'].update({'year':y})
            cash_flow_statement.append(statement['cash_flow_statement'])

In [ ]:
instruction = f"""
Consolidate the given data.

Each item in the given list is a balance sheet for a given year.

The keys need to be consolidated across the years.

You may leave some keys as None if there is no matching data for a given year.
However, try your best to leave as little data gaps as possible. If there is a
reported figure for a given year, it is likely in all years, or subsequent
years. For example, "assets" may change to "total_assets", but these should be
merged. There should be no nesting of dictionaries; the data for a given year
should be numbers.

Use your knowledge about the common items occurring in a balance sheet.

Your output should be json.loads ready.

Special instructions:
{special_instructions}

Data:
{balance_sheet}
"""
response = client.responses.create(
    model="o3-mini",
    input=instruction
)

In [ ]:
pd.DataFrame(json.loads(response.output_text)).set_index('year').to_csv(f"{root_path}\\balance_sheet.csv")

In [ ]:
instruction = f"""
Consolidate the given data.

Each item in the given list is an income statement for a given year.

The keys need to be consolidated across the years.

You may leave some keys as None if there is no matching data for a given year.
However, try your best to leave as little data gaps as possible. If there is a
reported figure for a given year, it is likely in all years, or subsequent
years. For example, "assets" may change to "total_assets", but these should be
merged. There should be no nesting of dictionaries; the data for a given year
should be numbers.

Use your knowledge about the common items occurring in an income statement.

Your output should be json.loads ready.

Special instructions:
{special_instructions}

Data:
{income_statement}
"""

response = client.responses.create(
    model="o3-mini",
    input=instruction
)

In [ ]:
pd.DataFrame(json.loads(response.output_text)).set_index('year').to_csv(f"{root_path}\\income_statement.csv")

In [ ]:
special_instructions = "Prefer VW group data over automotive group data."

instruction = f"""
Consolidate the given data.

Each item in the given list is a cash flow statement for a given year.

The keys need to be consolidated across the years.

You may leave some keys as None if there is no matching data for a given year.
However, try your best to leave as little data gaps as possible. If there is a
reported figure for a given year, it is likely in all years, or subsequent
years. For example, "assets" may change to "total_assets", but these should be
merged. There should be no nesting of dictionaries; the data for a given year
should be numbers.

Use your knowledge about the common items occurring in a cash flow statement.

Your output should be json.loads ready.

Special instructions:
{special_instructions}

Data:
{cash_flow_statement}
"""

response = client.responses.create(
    model="o3-mini",
    input=instruction
)

In [ ]:
pd.DataFrame(json.loads(response.output_text)).set_index('year').to_csv(f"{root_path}\\cash_flow_statement.csv")

# Data Visualization

In [ ]:
from matplotlib import pyplot as plt

In [ ]:
df_balance_sheet = pd.read_csv(f"{root_path}\\balance_sheet.csv", index_col=0)
df_income_statement = pd.read_csv(f"{root_path}\\income_statement.csv", index_col=0)
df_cash_flow_statement = pd.read_csv(f"{root_path}\\cash_flow_statement.csv", index_col=0)

## Balance Sheet

In [ ]:
df_balance_sheet.columns

In [ ]:
df_balance_sheet

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(12, 5))
df_balance_sheet['total_assets'].plot(label='Total Assets', ax=ax[0])
df_balance_sheet['current_assets'].plot(label='Current Assets', ax=ax[0])
df_balance_sheet['noncurrent_assets'].plot(label='Non-Current Assets', ax=ax[0])
df_balance_sheet[['current_liabilities', 'noncurrent_liabilities']].dropna().sum(axis=1).plot(label='Total Liabilities', ax=ax[1])
df_balance_sheet['current_liabilities'].dropna().plot(label='Current Liabilities', ax=ax[1])
df_balance_sheet['noncurrent_liabilities'].dropna().plot(label='Non-Current Liabilities', ax=ax[1])
ax[0].legend()
ax[1].legend()
fig.suptitle('Balance Sheet Data')

## Income Statement

In [ ]:
df_income_statement.columns

In [ ]:
df_income_statement

In [ ]:
df_income_statement['gross_profit'].plot(label='Gross Profit', title='Income Statement')
df_income_statement['sales_revenue'].plot(label='Total Revenue')
plt.legend()

## Cash Flow Statement

In [ ]:
df_cash_flow_statement.columns

In [ ]:
df_cash_flow_statement

In [ ]:
df_cash_flow_statement['cash_and_cash_equivalents_at_beginning_of_period'].dropna().plot()
df_cash_flow_statement['cash_and_cash_equivalents_at_end_of_period'].dropna().plot()